This notebook populates author annottations from pulication supplementary materials to the CAS object and outputs a new CAS json file.

In [12]:
# Install pip packages
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install cas-tools==0.0.1.dev47


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: /Users/aa37/Documents/GitHub/siletti_hacking/venv/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: /Users/aa37/Documents/GitHub/siletti_hacking/venv/bin/python3 -m pip install --upgrade pip


In [13]:
import os
import json
import random
import anndata
import pandas as pd
import urllib.request as request

from cas.file_utils import read_json_file, write_dict_to_json_file
from cas.add_author_annotations import add_author_annotations

Read CAS json

In [14]:
cas = read_json_file('./intermediatte_files/Siletti_from_abc.json')

Read supplementary materials table 3 from https://www.science.org/doi/10.1126/science.add7046

In [15]:
# Load the CSV file
csv_df = pd.read_csv('./Author_cathegories/science.add7046_table_s3.csv')
# filter empty rows
csv_df = csv_df[csv_df['cell_label'].notna()]
csv_df.head(3)

,Cluster ID,cell_label,Supercluster,Class auto-annotation,Neurotransmitter auto-annotation,Neuropeptide auto-annotation,Subtype auto-annotation,Transferred MTG Label,Top three regions,Top three dissections,...,DoubletFinder score,Total UMI,Fraction unspliced,Fraction mitochondrial,H19.30.002,H19.30.001,H18.30.002,H18.30.001,Fraction cells from top donor,Number of donors
0,0.0,Bcell_0,Miscellaneous,BCELL,NaN,NaN,NaN,NaN,"Midbrain: 21.0%, Basal forebrain: 19.0%, Pons:...","Human GPi: 11.4%, Human SN: 11.4%, Human SN-RN...",...,0.013358,2258.866667,0.608113,0.020989,58.0,20.0,27.0,0.0,0.552381,3.0
1,1.0,Tcell_1,Miscellaneous,TCELL,NaN,NaN,NaN,NaN,"Midbrain: 15.0%, Basal forebrain: 14.0%, Pons:...","Human SpC: 8.2%, Human CbDN: 7.6%, Human SN: 7.4%",...,0.022371,2184.314470,0.582884,0.021212,708.0,400.0,1364.0,2.0,0.551334,4.0
2,2.0,Nkcell_2,Miscellaneous,NK,NaN,NaN,NaN,NaN,"Thalamus: 17.6%, Cerebral cortex: 15.5%, Basal...","Human GPe: 5.7%, Human GPi: 5.5%, Human CbDN: ...",...,0.023788,2263.348733,0.558228,0.020093,245.0,126.0,300.0,0.0,0.447094,3.0


In [16]:
csv_columns = csv_df.columns.tolist()
exclude_columns = ["Cluster ID", "cell_label", "Supercluster"]
columns_to_ingest = [column for column in csv_columns if column not in exclude_columns]
columns_to_ingest

['Class auto-annotation',
 'Neurotransmitter auto-annotation',
 'Neuropeptide auto-annotation',
 'Subtype auto-annotation',
 'Transferred MTG Label',
 'Top three regions',
 'Top three dissections',
 'Top Enriched Genes',
 'Number of cells',
 'DoubletFinder score',
 'Total UMI',
 'Fraction unspliced',
 'Fraction mitochondrial',
 'H19.30.002',
 'H19.30.001',
 'H18.30.002',
 'H18.30.001',
 'Fraction cells from top donor',
 'Number of donors']

In [17]:
# cell_label is required, add it forcefully if not exist
for annotation in cas["annotations"]:
    if "cell_label" not in annotation:
        print(annotation)
        annotation["cell_label"] = "NA"

{'labelset': 'neurotransmitter', 'cell_set_accession': 'CS202210140_3826'}


In [18]:
updated_cas = add_author_annotations(cas, csv_df, "cell_label", columns_to_ingest)

In [19]:
write_dict_to_json_file('./intermediatte_files/Siletti_added_author_categories_v2.json', updated_cas)